In [1]:
# Imports
from dask_jobqueue import SLURMCluster
from distributed import Client
from BD_causalflow_gauss import run as run_flows_gaus
from BD_causalflow_laplace import run as run_flows_laplace
from BD_causalflow_ta import run as run_flows_tailadaptive

In [2]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10095",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
    #job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=100)
client = Client(cluster)

In [3]:
# Submitting jobs
n = 1000
ntrial = 20
base = "MixedTails"
futures = []
for seed in range(ntrial):
    f2 = client.submit(run_flows_gaus,seed,n, base = base)
    f3 = client.submit(run_flows_laplace,seed,n, base = base)
    f4 = client.submit(run_flows_tailadaptive,seed,n, base = base)
    futures += [f2,f3,f4]

In [4]:
futures

[<Future: pending, key: run-51d863a1d525824330b8d6c2a9e7d796>,
 <Future: pending, key: run-a0765cfb8344de44b06ca9a9d9c5496a>,
 <Future: pending, key: run-f913d44f8990040229e7b8dc4c5d8313>,
 <Future: pending, key: run-3145be8d54ddb6e2985efe5af0322d4d>,
 <Future: pending, key: run-0df315d859465d6aa8721444ecb44bd7>,
 <Future: pending, key: run-3af64f32f8e6c489c97cdf808aecf46b>,
 <Future: pending, key: run-7e4a37472dee62fb9f9a12fd5125126b>,
 <Future: pending, key: run-26687ae6fb524804828ccba4eb620af6>,
 <Future: pending, key: run-ba16c02f9f7a212fe158c744ef30bd60>,
 <Future: pending, key: run-d5dfbbd12e83bbf350e79617dfa44204>,
 <Future: pending, key: run-3f02a886d2ab2313a10e7da4b0d56a2a>,
 <Future: pending, key: run-dee827e16f819d1944a43d3c3198917f>,
 <Future: pending, key: run-1644f83664998a4d40f589666296fcc6>,
 <Future: pending, key: run-12c7768c1555232d816c49f25d4b236b>,
 <Future: pending, key: run-2366dda7c4ab5a7cb2e05b795b0c50d8>,
 <Future: pending, key: run-6385a7dceb771893b924dc9ae4b

In [5]:
# Getting results
results = client.gather(futures)

In [6]:
# Closing client
client.close()
cluster.close()

In [7]:
import torch
torch.save(f = "BD_results_cflows_trials={0}_n={1}_base={2}.pt".format(ntrial,n,base), obj = results)